# Report

Goal: Vary X values and compare measure the system performance Y

Three core economic categories of variables:

### 1. Order Flow (Market Activity)

These variables control the frequency and nature of orders hitting the market, which translates to **risk** and **demand**.

-   `lam_mkt_buy` (or `lam_mkt_sell`): $\lambda_{mkt}$
    -   **Liquidity Demand / Volatility**
    -   **Pairing:** With MM Strategy (e.g., `mm_base_spread`) to see how the MM's **required spread** and **PnL** react to changes in raw market order intensity.
-   `lam_limit_buy` (or `lam_limit_sell`): $\lambda_{limit}$
    -   **Liquidity Supply / Depth**
    -   **Pairing:** With Market Structure (e.g., `limit_price_decay_rate`) to see how the **quantity of resting orders** interacts with the **distribution** (how close orders are to the best price).
-   `order_size_mean`: $\lambda_{\text{size}}$
    -   **Adverse Selection / Order Risk**
    -   **Pairing:** With MM Strategy (e.g., `mm_base_spread` or `mm_skew_coef`) to find the **risk premium** required to compensate the MM for facing large, potentially informed orders. (Highly Recommended)


### 2. Market Structure (LOB Configuration)

These variables define the physical shape and stability of the Limit Order Book (LOB).

-   `limit_price_decay_rate`: $\alpha$
    -   **Liquidity Depth / Price Distribution**
    -   **Pairing:** With Order Flow (e.g., `order_size_mean`) to analyze how a **shallow LOB** (high decay) affects the MM's **losses** when hit by large market orders. (Highly Recommended)
-   `lam_cancel`: $\lambda_{cancel}$
    -   **Order Book Stability / Fickleness**
    -   **Pairing:** With MM Strategy (e.g., `mm_skew_coef`) to see if constant order **cancellations** make it harder for the MM to control their **inventory** (inventory risk/volatility).


### 3. Market Maker Strategy

These variables control the automated quoting logic of the Market Maker.

-   `mm_base_spread`: $S_{base}$
    -   **MM Competitiveness / Risk Premium**
    -   **Pairing:** With Risk (e.g., `order_size_mean`) to determine the **optimal base spread** that maximizes the MM's **PnL** while balancing the risk of large trades.
-   `mm_skew_coef`: $\beta$
    -   **Inventory Risk Management Strategy**
    -   **Pairing:** With Order Flow/Structure (e.g., `lam_mkt_buy` or `limit_price_decay_rate`) to measure the **effectiveness** of the MM's automated inventory control in different market conditions.

---

## Performance Metrics by System Variable Category

### A. Order Flow Variables

(Governed by: $\lambda_{limit\_buy}$, $\lambda_{limit\_sell}$, $\lambda_{mkt\_buy}$, $\lambda_{mkt\_sell}$, $\lambda_{cancel}$, `order_size_mean`)

These variables define the market environment created by the "noise" traders. The metrics measure the resulting stability and depth of the market.

| Focus Area | Metric to Record | Rationale |
| :--- | :--- | :--- |
| **Liquidity** | **Average Bid-Ask Spread** | Measures the *transaction cost* of immediacy. Higher order arrival rates ($\lambda_{limit}$) should decrease this, while higher consumption rates ($\lambda_{mkt}$) might increase it. |
| **Liquidity** | **LOB Depth** (e.g., Average Volume within $\pm 5$ ticks) | Measures the amount of standing limit orders, which is directly related to $\lambda_{limit}$ and `order_size_mean`. Deep liquidity buffers the price against large market orders. |
| **Liquidity** | **Resting Execution Times** (Mean, Median, P99) | Measures the time it takes for a limit order to be filled. High $\lambda_{mkt}$ decreases this; high $\lambda_{limit}$ increases it. |
| **Volatility/Price Discovery** | **Trade Count / Trades Per Hour** | Measures the activity level and throughput of the market. Directly reflects the combined effect of $\lambda_{mkt}$ rates. |
| **Volatility/Price Discovery** | **Price Volatility** (e.g., Variance of Mid-Price Returns) | Measures the stability of the price process. High $\lambda_{mkt}$ and low LOB depth can lead to higher volatility. |
| **Trader Behavior** | **Resting vs. Market Execution Ratio** | Measures the ratio of liquidity supplied (resting limit orders) versus liquidity demanded (market orders). Affected by the balance of $\lambda_{limit}$ and $\lambda_{mkt}$. |

---

### B. Market Structure Variables

(Governed by: `limit_price_decay_rate`, `initial_price`, `sim_duration`)

These variables shape the foundational characteristics of the Order Book. The metrics should focus on the quality of liquidity provision and efficiency.

| Focus Area | Metric to Record | Rationale |
| :--- | :--- | :--- |
| **Liquidity** | **LOB Depth at specific levels** (e.g., $1^{\text{st}}$, $2^{\text{nd}}$, $3^{\text{rd}}$ tick)  | **Crucial** for `limit_price_decay_rate`. This measures *where* the liquidity resides. A low decay rate will result in a flatter depth profile, while a high decay rate will show a large volume at the Best Bid/Ask and then a sharp drop-off. |
| **Liquidity** | **Tick-to-Tick Spread** | Measures how often the spread is exactly one tick. A well-functioning LOB with a high decay rate should have a tight, often one-tick, spread. |
| **Efficiency** | **Price Movement per Unit Volume** (Effective Spread) | Measures market impact: how much the price moves for a given trade size. This is inversely related to overall depth (which is heavily influenced by `limit_price_decay_rate`). |
| **Efficiency** | **Simulation Run Time** (for comparison/scaling) | While not a financial metric, recording the computation time relative to `sim_duration` is useful for assessing simulation efficiency and scaling. |

---

### C. Market Maker Strategy Variables

(Governed by: `mm_base_spread`, `mm_skew_coef`, `mm_quote_interval`)

These variables govern the Market Maker's profit/loss and risk management. The metrics focus on the MM's success and their impact on market quality.

| Focus Area | Metric to Record | Rationale |
| :--- | :--- | :--- |
| **MM Profitability** | **MM Final PnL** (Total, Per Hour, Per Trade) | The ultimate measure of the strategy's success. Higher `mm_base_spread` generally leads to higher PnL, but higher `mm_skew_coef` may reduce PnL if it pushes quotes too far from the best price. |
| **MM Risk Management** | **MM Final/Average Inventory** 

[Image of Market Maker Inventory Over Time]
 | Measures how effectively the MM controls their risk exposure. A good strategy (e.g., effective `mm_skew_coef`) should keep the average inventory close to zero. |
| **MM Risk Management** | **MM Realized Spread** | Measures the profit the MM actually makes per round trip, calculated as the difference between the mid-price at execution and the quoted price. This reveals adverse selection risk. |
| **Liquidity Impact** | **MM Contribution to Depth** | Measures the percentage of volume at the Best Bid/Ask that is placed by the Market Maker. Directly affected by all three MM parameters. |

---

For
1. Order strategy:
   - We chose `order_size_mean`
2. Market Structure:
   - Choose `limit_price_decay_rate`
3. MM Strat:
   - Chose `base spread`